In [1]:
from elasticsearch import Elasticsearch
from deepface import DeepFace
import matplotlib.pyplot as plt
from retinaface import RetinaFace
import os
import time
import glob

ModuleNotFoundError: No module named 'elasticsearch'

In [ ]:
es = Elasticsearch(hosts=['http://localhost:9200'], http_auth=('elastic', 'DkIed99SCb'))

In [ ]:
model_name = 'ArcFace'
target_size = (160, 160)
embedding_size = 512

In [ ]:
mapping = {
    "mappings": {
        "properties": {
            "title_vector":{
                "type": "dense_vector",
                "dims": embedding_size
            },
            "title_name": {"type": "keyword"}
        }
    }
}
   
es.indices.create(index="policedb2", body=mapping)

In [ ]:
dir_path = "dataset/police/*.*"
res = glob.glob(dir_path, recursive=True)
files = []
for item in res:
    if item.endswith('.jpg'):
        files.append(item)
print('total files:', len(files))

In [ ]:
index = 0
tic = time.time()

for img_path in files:
    print('indexing', img_path)
    try:
        embedding_objs = DeepFace.represent(img_path=img_path, 
            model_name = model_name,
            detector_backend = 'retinaface'
        )
        embedding = embedding_objs[0]["embedding"]
        doc = {"title_vector": embedding, "title_name": img_path}
        es.create("policedb2", id=index, body=doc)
    except:
        pass

    index = index + 1
toc = time.time()
print("indexing completed in ",toc-tic," seconds")

In [ ]:
target_img_path = "dataset/targets/5.jpeg"
embedding_objs = DeepFace.represent(img_path = target_img_path, 
  model_name = model_name,
  detector_backend = 'retinaface'
)
target_embedding = embedding_objs[0]["embedding"]

In [ ]:
query = {
    "size": 5,
    "query": {
    "script_score": {
        "query": {
            "match_all": {}
        },
        "script": {
            #"source": "cosineSimilarity(params.queryVector, 'title_vector') + 1.0",
            "source": "1 / (1 + l2norm(params.queryVector, 'title_vector'))", #euclidean distance
            "params": {
                "queryVector": list(target_embedding)
            }
        }
    }
}}

In [ ]:
tic = time.time()
res = es.search(index="policedb2", body=query)
toc = time.time()

print(toc-tic," seconds")

In [ ]:
target_faces = RetinaFace.extract_faces(img_path = target_img_path, align = True)
target_img = target_faces[0]

for i in res["hits"]["hits"]:
    candidate_name = i["_source"]["title_name"]
    candidate_score = i["_score"]
    print(candidate_name, ". Score: ", candidate_score)
    candidate_faces = RetinaFace.extract_faces(img_path = candidate_name, align = True)
    candidate_img = candidate_faces[0]

    fig = plt.figure()

    ax1 = fig.add_subplot(1, 2, 1)
    plt.imshow(target_img)
    plt.axis('off')

    ax2 = fig.add_subplot(1, 2, 2)
    plt.imshow(candidate_img)
    plt.axis('off')

    plt.show()

    print("-------------------------")